In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn as nn
from sklearn import datasets

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_test_data= [train, test] 
for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

title_mapping = {"Mr": "Mr", "Miss": "Miss", "Mrs": "Mrs", 
                 "Master": "etc", "Dr": "etc", "Rev": "etc", "Col": "etc", "Major": "etc", "Mlle": "etc","Countess": "etc",
                 "Ms": "etc", "Lady": "etc", "Jonkheer": "etc", "Don": "etc", "Dona" : "etc", "Mme": "etc","Capt": "etc","Sir": "etc" }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    
train.drop('Name', axis=1, inplace=True)
test.drop('Name', axis=1, inplace=True)

In [4]:
train.drop('Ticket', axis=1, inplace=True)
test.drop('Ticket', axis=1, inplace=True)

test_PassengerId=test["PassengerId"]
train.drop('PassengerId', axis = 1, inplace=True)
test.drop('PassengerId', axis = 1, inplace=True)

In [5]:
# Cabin 앞글자만 따기
train['Cabin'] = train['Cabin'].str[:1]

# mapping
cabin_mapping = {"A": 0, "B": 0, "C": 0, "D": 0, "E": 0, "T": 0, "F": 1, "G": 1}

for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)

In [6]:
# fillna
train['Cabin'].fillna((train['Pclass'] == 1) * 1, inplace=True)
test['Cabin'].fillna((test['Pclass'] == 1) * 1, inplace=True)

train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)
test["Age"].fillna(test.groupby("Title")["Age"].transform("median"), inplace=True)

train.replace({"Fare": {0: np.nan}}, inplace=True)
test.replace({"Fare": {0: np.nan}}, inplace=True)

train["Fare"].fillna(train.groupby("Pclass")["Fare"].transform("median"), inplace=True)
test["Fare"].fillna(test.groupby("Pclass")["Fare"].transform("median"), inplace=True)

for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [7]:
# binning
for dataset in train_test_data:
    dataset.loc[ dataset['Age'] <= 10, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 10) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 34), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 34) & (dataset['Age'] <= 46), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 46) & (dataset['Age'] <= 62), 'Age'] = 5
    dataset.loc[ dataset['Age'] > 62, 'Age'] = 6

    
for dataset in train_test_data:
    dataset.loc[ dataset['Fare'] <= 10, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 10) & (dataset['Fare'] <= 30), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3

In [8]:
# 머신에게 학습을 시킬때는 문자열로는 학습이 안된다. 모든 값을 숫자로 매핑시킨다.
sex_mapping = {"male": 0, "female": 1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

In [9]:
# 선착장을 숫자로 맵핑
embarked_mapping = {"S": 0, "C": 1, "Q": 2}
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

In [10]:
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, "etc": 3}
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)

In [11]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,0,2.0,1,0,0.0,0.0,0,0
1,1,1,1,4.0,1,0,2.0,0.0,1,2
2,1,3,1,3.0,0,0,0.0,0.0,0,1
3,1,1,1,4.0,1,0,2.0,0.0,0,2
4,0,3,0,4.0,0,0,0.0,0.0,0,0


In [12]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,3,0,4.0,0,0,0.0,0.0,2,0
1,3,1,5.0,1,0,0.0,0.0,0,2
2,2,0,5.0,0,0,0.0,0.0,2,0
3,3,0,3.0,0,0,0.0,0.0,0,0
4,3,1,2.0,1,1,1.0,0.0,0,2


In [13]:
train_data = train.drop('Survived', axis=1)
train_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,3,0,2.0,1,0,0.0,0.0,0,0
1,1,1,4.0,1,0,2.0,0.0,1,2
2,3,1,3.0,0,0,0.0,0.0,0,1
3,1,1,4.0,1,0,2.0,0.0,0,2
4,3,0,4.0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...
886,2,0,3.0,0,0,1.0,0.0,0,3
887,1,1,2.0,0,0,1.0,0.0,0,1
888,3,1,2.0,1,2,1.0,0.0,0,1
889,1,0,3.0,0,0,1.0,0.0,1,0


In [14]:
train_label = train['Survived']
train_label

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [15]:
input_size = 9
hidden_size = 56
num_classes = 2
num_epochs = 100
learning_rate = 0.01

In [16]:
class NeuralNet(nn.Module): 
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    # 모델의 Forward Path를 정의
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
    
        return out

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, 
                                                    train_label, 
                                                    random_state=42,
                                                    stratify=train_label)
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

In [18]:
# 위에서 정의한 클래스를 인스턴스화 시킴
# model = NeuralNet(input_size, hidden_size, num_classes).to(device) # to(device) : 이 모델을 gpu 서버에서 돌린다는 뜻
model = NeuralNet(input_size, hidden_size, num_classes)
# loss, optimizer를 선정의
loss_function = nn.CrossEntropyLoss() # Loss 기능 안에 Softmax 함수 기능 포함되어져 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_list = []

for epoch in range(num_epochs): # 5번
    # Forward Pass
    pred = model(X_train)
    loss = loss_function(pred, y_train)
    
    loss_list.append(loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch : [{epoch + 1}/{num_epochs}], Loss : {loss.item():.4f}")

Epoch : [10/100], Loss : 0.4548
Epoch : [20/100], Loss : 0.4254
Epoch : [30/100], Loss : 0.4054
Epoch : [40/100], Loss : 0.3913
Epoch : [50/100], Loss : 0.3802
Epoch : [60/100], Loss : 0.3703
Epoch : [70/100], Loss : 0.3618
Epoch : [80/100], Loss : 0.3533
Epoch : [90/100], Loss : 0.3459
Epoch : [100/100], Loss : 0.3395


In [19]:
with torch.no_grad(): # 미분 안하겠다...실제로 학습할 필요가 없을 때 이 구문을 반드시 작성
    correct = 0
    total = 0

    for features, labels in zip(X_test, y_test):
        outputs = model(features)
        
        total += 1
        correct += (torch.argmax(outputs) == labels).sum().item()

    print(f"Accuracy of the Network on the Test Images : {100*correct/total}%")

Accuracy of the Network on the Test Images : 78.47533632286995%


### Submit

In [20]:
train_data = torch.FloatTensor(train_data.values)
test = torch.FloatTensor(test.values)
train_label = torch.LongTensor(train_label.values)

In [21]:
# 위에서 정의한 클래스를 인스턴스화 시킴
# model = NeuralNet(input_size, hidden_size, num_classes).to(device) # to(device) : 이 모델을 gpu 서버에서 돌린다는 뜻
model = NeuralNet(input_size, hidden_size, num_classes)
# loss, optimizer를 선정의
loss_function = nn.CrossEntropyLoss() # Loss 기능 안에 Softmax 함수 기능 포함되어져 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs): # 100번
    # Forward Pass
    pred = model(train_data)
    loss = loss_function(pred, train_label)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch : [{epoch + 1}/{num_epochs}], Loss : {loss.item():.4f}")

Epoch : [10/100], Loss : 0.4881
Epoch : [20/100], Loss : 0.4436
Epoch : [30/100], Loss : 0.4198
Epoch : [40/100], Loss : 0.4046
Epoch : [50/100], Loss : 0.3934
Epoch : [60/100], Loss : 0.3846
Epoch : [70/100], Loss : 0.3780
Epoch : [80/100], Loss : 0.3726
Epoch : [90/100], Loss : 0.3683
Epoch : [100/100], Loss : 0.3641


In [22]:
with torch.no_grad(): # 미분 안하겠다...실제로 학습할 필요가 없을 때 이 구문을 반드시 작성
    correct = 0
    total = 0
    
    prediction = []
    
    for features in test:
        outputs = model(features)
        prediction.append(torch.argmax(outputs).item())

In [23]:
submission = pd.read_csv('../submit/gender_submission.csv')
# submission

In [24]:
# 예측한 결과를 집어넣어 다시 csv파일로 저장한다.
submission['Survived'] = prediction
submission.to_csv('../submit/gender_submission.csv', index=False)